### 1. Data Review & Cleaning

#### 1.1. Initial review - remove redundant columns, standardize column names

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Read dataset
data = pd.read_csv("fifa21_male2.csv")

In [ ]:
# Have a quick look at the data:
data.head()

We notice immediately that the columns `Player Photo`, `Club Logo`, and `Flag Photo` contain *sofifa* links, which will not help in the current analysis, so we can drop them.

In [ ]:
data.drop(['Player Photo', 'Club Logo', 'Flag Photo'], axis=1, inplace=True)

We also notice there is an ID column which we can use as our index column, after ensuring it doesn't have any duplicate values:

In [ ]:
print(len(data['ID'].unique())/data.shape[0]) # equal to 1, so no duplicates

In [ ]:
data.set_index('ID', inplace=True, drop=True)

We will also remove the `Name` column, as it's unlikely to be relevant to our analysis and store it in a different dataframe for future reference.

In [ ]:
names = data['Name']
data.drop('Name', axis=1, inplace=True)

We also notice there are many columns names which are abbreviations - so we will rename those to the full name for clarity. To do so, we are using a `.csv` file with the abbreviations and their corresponding meaning, as seen on the *sofifa* website. Alongside, we will also standardize the column names:

In [ ]:
# Change capitalization to lowercase and replace spaces with underscores:
data.columns = data.columns.str.lower()
data.columns = data.columns.str.replace(" ", "_")

In [ ]:
# Read positions.csv into dataframe
positions = pd.read_csv('positions.csv', header=None, index_col=0)

# Change dataframe to series and then dictionary so it can be used to rename columns:
positions = positions.squeeze().to_dict()

# Change column names:
for column in data.columns:
    if column in positions:
        data.rename(columns=positions, inplace=True)

Now, we can quickly check which columns have more than 75% null values, so we can discard them from our analysis:

In [ ]:
def check_null_values(df, threshold=75):
    nulls_percentage = {}
    for column in df.columns:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        if null_percentage >= threshold:
            nulls_percentage[column] = null_percentage
    return nulls_percentage

check_null_values(data)

In [ ]:
# As `loan_date_end` has mostly `NaN` values, we can discard it:
data.drop('loan_date_end', axis=1, inplace=True)

To get a sense of the amount of null values in our data, we can check the maximum percentage of `NaN` values in the dataframe:

In [ ]:
def max_nulls(df):
    nulls_percentage = []
    for column in df.columns:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        nulls_percentage.append(null_percentage)
    return max(nulls_percentage)

max_nulls(data)

Given the maximum amount of nulls is low (2.5%), we can continue with our initial review and get back to replacing null values later. 

Next, we'll review the number of unique values per column and ensure there are no columns with one value only, as they will not add any value to the analysis:

In [ ]:
def check_unique_values(df):
    single_value_columns = []
    for column in df:
        if len(df[column].unique()) == 1:
            single_value_columns.append(column)
    return single_value_columns

check_unique_values(data)

In [ ]:
# Check the unique values in the gender column
data['gender'].unique()

In [ ]:
# Remove the gender column as the data shows only male players
data.drop('gender', axis=1, inplace=True)

After looking once more at the data, we observe that the `team_&_contract` column seems to have similar information to the `club` & `contract` columns, so we might be able to remove it too. However, we first have to check our assumption is correct.

In [ ]:
# Create a joined column to compare to the team & contract column
for i in range(0, data.shape[0]):
    if data['contract'].iloc[i] == np.nan:
        data['contract'].iloc[i] = ' '
    if data['club'].iloc[i] == np.nan:
        data['club'].iloc[i] = ' '

data['club_&_contract'] = data['club'] + ' ' + data['contract']

In [ ]:
# Check that the columns are identical and save non-identical values to a dataframe
def check_identical_columns(col1, col2, df):
    diff_values = pd.DataFrame({col1:[], col2:[]})
    identical = 1

    for i in range(0, df.shape[0]):
        if df[col1].iloc[i] == df[col2].iloc[i]:
            continue
        else:
            diff_values.loc[len(diff_values.index)] = [df[col1].iloc[i], df[col2].iloc[i]]
            identical = 0
            

    if identical == 0:
        return diff_values
    else:
        return 'Columns are identical.'         
            
diff_values = check_identical_columns(col1='club_&_contract', col2='team_&_contract', df=data)        

diff_values

We notice two things:
- the `club_&_contract` column has null values, whereas the `team_&_contract` column doesn't
* the `club_&_contract` rows that are non-identical also contain the country that the club belongs to

As the number of values containing the country are low compared to the number of rows in the dataframe, we'll quickly check if the null values come from the `club` and / or `contract` column to see if we can discard the `team_&_contract` column:

In [ ]:
# Check the rows with null values
diff_values[diff_values['club_&_contract'].isna()]

# Check if the nulls come from the contract or club columns:
print(data['contract'].isna().sum())
print(data['club'].isna().sum())

It seems that the null values in the `club_&_contract` column came from the `club` column. However, given that the `team_&_contract` values corresponding to those rows also contain only the contract information, we can remove it, as it doesn't provide any additional information.

In [ ]:
data.drop(['team_&_contract', 'club_&_contract'], axis=1, inplace=True)

#### 1.2. Data Cleaning

##### 1.2.1. Numerical Data Cleaning

`value`, `wage`, and `release clause` columns:

In [ ]:
financials = ['value', 'wage', 'release_clause']

def clean_value(i):
    x = float(i.replace(".","").replace("€","").replace("K","000").replace("M","00000"))
    return x

for column in financials:
    data[column] = data[column].apply(clean_value)

`weight` column:

In [ ]:
def clean_weight(i):
    x = float(i.replace('lbs',''))
    return x

data["weight"] = data["weight"].apply(clean_weight)

`height` column:

In [ ]:
def convert_height(i):
    to_cm = 2.54
    x = i.replace("'"," ")
    x2 = x.replace('"','') # need to do it in two parts 
                           # because of different quote used 
                           # for inch and foot 
    y = x2.split()
    height = round(((float(y[0])*12)+float(y[1]))*to_cm,0)
    return height

data['height'] = data['height'].apply(convert_height)
data['height']

Cleaning the positions columns, i.e. `left-striker`, `goalkeeper`, etc. :

In [ ]:
def cleaning_positions(i):
    x = float(i.replace("+",".").replace("-",""))
    return x

for col in data.loc[:, 'left_striker':'goalkeeper']:
    data[col] = data[col].apply(cleaning_positions)
data.head()

In addition to cleaning the data within, we will also find which columns have identical scores:

In [ ]:
# We'll first extract the names of the positions that have players on the right / left / center
first_position = data.columns.get_loc('left_striker') 
last_position = data.columns.get_loc('goalkeeper')

position_names = []

for i in range(first_position, last_position + 1):
    column_name = data.columns[i]
    if 'left' in column_name:
        position_names.append(column_name[5:])

print(position_names)

In [ ]:
# Now we'll find the groups that we'd like to check
groups = {}

for position in position_names:
    placements = []
    for i in range(first_position, last_position + 1):
        column_name = data.columns[i]
        if position in column_name:
            placements.append(column_name)
    groups[position] = placements

# Remove the extra values from the 'back' list:
groups['back'] = ['left_back', 'center_back', 'right_back']

In [ ]:
# Find identical columns and store the indexes in a list to later drop
to_drop = []

for position in groups.keys():
    positions_number = len(groups[position])
    for i in range(0, positions_number):
        for j in range(i + 1, positions_number):
            column_1 = groups[position][i]
            column_2 = groups[position][j]
            if data[column_1].equals(data[column_2]):
                if column_2 not in to_drop:
                    to_drop.append(column_2)

# Drop redundant columns
data.drop(to_drop, axis=1, inplace=True)

In [ ]:
# Remove the 'left_' string from the remaining columns
for column in data.columns:
    if 'left' in column:
        data.rename({column: column[5:]}, axis=1, inplace=True)

data.head()

`weak_foot`, `skill_moves`, and `international_reputation` columns:

In [ ]:
star_columns = ['weak_foot', 'skill_moves', 'international_reputation']

# Check unique values
for column in star_columns:
    print(data[column].unique())

In [ ]:
# Select only the first character from the string, then convert the data type to integer
# and check the operation was successful
for column in star_columns:
    data[column] = data[column].str[0]
    data[column] = pd.to_numeric(data[column], errors='raise')
    print(data[column].dtypes)

##### 1.2.2. Categorical Data Cleaning

Change the `positions` data to the number of possible positions:

In [ ]:
def clean_positions(x):
    number_of_positions = str(x).split() 
    position = len(number_of_positions) 
    return position 

data['position'] = data['position'].apply(clean_positions)

Extract the year from the `joined` column:

In [ ]:
# As the end of the joined is typically represented by the last 4 characters of the 
# strings, we will extract those where possible:
def extract_year(x):
    try:
        x = int(x[-4:])
    except:
        pass
    return x

data['joined'] = data['joined'].apply(extract_year)

We will use the same function to extract the end year from the `contract` column:

In [ ]:
# Explore the contract data
data['contract'].unique()

# Use extract year function to get the year
data['contract'] = data['contract'].apply(extract_year)

# Check what non-integer values remained in the column:
data['contract'].unique()

The values having `On Loan` at the end mean that the player still has a contract with the club, but he's borrowed to a different team in the meantime. We can create a separate column to track the players that have been on loan:

In [ ]:
def record_loans(x):
    if type(x) == str and 'On Loan' in x:
        record = 'Yes'
    else:
        record = 'No'
    return record    

data['on_loan'] = data['contract'].apply(record_loans)

# Check the operation was successful
data['on_loan'].unique()

In [ ]:
# We'll now remove the 'On Loan' string from the contract column to extract the year:
def clean_loans(x):
    try:
        x = int(x.replace(" On Loan", "")[-4:])
    except:
        pass
    return x    

data['contract'] = data['contract'].apply(clean_loans)

# Check the operation was successful
data['contract'].unique()

We notice the only odd values left are the ones looking like *Country Free* and the *1648*, so we will look at how many rows have these type of values:

In [ ]:
odd_ones_out = []

for value in data['contract']:
    if type(value) == str and 'Free' in value:
        odd_ones_out.append(value)
    elif value == 1648:
        odd_ones_out.append(value)

len(odd_ones_out)

As there are very few odd values compared to the number of rows in the dataset, we can replace them with nulls:

In [ ]:
def clean_odds(x):
    if x in odd_ones_out:
        x = np.nan
    return x    

data['contract'] = data['contract'].apply(clean_odds)

# Rename column to end_contract
data.rename({'contract': 'end_contract'}, axis=1, inplace=True)

# Check the operation was successful
data['end_contract'].unique()

`nationality` column:

In [ ]:
data["nationality"] = data["nationality"].apply(lambda x: "Democratic Republic of the Congo" if str(x).startswith("DR")
                                                 else "North Korea" if str(x).endswith("DPR")
                                                 else "China" if str(x).endswith("PR")
                                                 else str(x).replace("&amp;","and") if "&amp;" in x
                                                 else x)

In [ ]:
data.head()

We will also check if there is any numerical data still stored as an object:

In [ ]:
for col in data.columns:
    print(col, ":", data[col].dtypes)

As the `hits` column is still an object, we will change it to numerical data:

In [ ]:
data['hits'] = pd.to_numeric(data['hits'], errors='coerce')

#### 1.3. Find & replace null values

In [ ]:
# Check there aren't any rows with too many null values:
def check_null_values_rows(df, threshold=75):
    nulls_percentage = {}
    for index in df.index:
        number_of_nulls = df.loc[index,].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[1], 1)
        if null_percentage >= threshold:
            nulls_percentage[index] = null_percentage
    return nulls_percentage

check_null_values_rows(data)

As there are no rows with more than 75% `NaN` values, we can begin replacing these values per column. Before doing so, however, we will also check the maximum amount of nulls we have in any given row:

In [ ]:
def max_nulls_rows(df):
    nulls_percentage = []
    for index in df.index:
        number_of_nulls = df.loc[index,].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[1], 1)
        nulls_percentage.append(null_percentage)
    return max(nulls_percentage)

max_nulls_rows(data)

As the maximum amount of null values within a row is only ~15%, we can move on to replacing the null values in the relevant columns.

In [ ]:
# Check number of null values
for col in data.columns:
    if data[col].isna().sum() != 0:
        print(col, ":", round((data[col].isna().sum()) * 100 / data.shape[0], 0),"%")

In [ ]:
# Max left is 2% so we can safely drop the remaining NaN values:
data = data.dropna()

#### 1.4. Exploratory Data Analysis

##### 1.4.1. Categorical & Discrete Numerical data

In [ ]:
# Select categorical data
cat = data.select_dtypes(np.object)

# Show bar plots for categorical data with few values
for column in cat:
    x = cat[column].unique()
    y = cat[column].value_counts()
    if len(x) < 100:
        fig, ax = plt.subplots(figsize = (12, 9))
        plt.title(column)
        plt.bar(x, y)
        plt.show()
